# ADS 509 Module 1: APIs and Web Scraping

This notebook has two parts. In the first part, you will scrape lyrics from AZLyrics.com. In the second part, you'll run code that verifies the completeness of your data pull. 

For this assignment you have chosen two musical artists who have at least 20 songs with lyrics on AZLyrics.com. We start with pulling some information and analyzing them.


## General Assignment Instructions

These instructions are included in every assignment, to remind you of the coding standards for the class. Feel free to delete this cell after reading it. 

One sign of mature code is conforming to a style guide. We recommend the [Google Python Style Guide](https://google.github.io/styleguide/pyguide.html). If you use a different style guide, please include a cell with a link. 

Your code should be relatively easy-to-read, sensibly commented, and clean. Writing code is a messy process, so please be sure to edit your final submission. Remove any cells that are not needed or parts of cells that contain unnecessary code. Remove inessential `import` statements and make sure that all such statements are moved into the designated cell. 

Make use of non-code cells for written commentary. These cells should be grammatical and clearly written. In some of these cells you will have questions to answer. The questions will be marked by a "Q:" and will have a corresponding "A:" spot for you. *Make sure to answer every question marked with a `Q:` for full credit.* 


# Importing Libraries

In [1]:
import os
import datetime
import re

In [2]:
# for the lyrics scrape section
import requests
import time
from bs4 import BeautifulSoup
from collections import defaultdict, Counter
import random



---

# Lyrics Scrape

This section asks you to pull data by scraping www.AZLyrics.com. In the notebooks where you do that work you are asked to store the data in specific ways. 

In [3]:
artists = {'taylorswift':"https://www.azlyrics.com/t/taylorswift.html",
           'blackeyedpeas':"https://www.azlyrics.com/b/blackeyedpeas.html"} 
# we'll use this dictionary to hold both the artist name and the link on AZlyrics

## A Note on Rate Limiting

The lyrics site, www.azlyrics.com, does not have an explicit maximum on number of requests in any one time, but in our testing it appears that too many requests in too short a time will cause the site to stop returning lyrics pages. (Entertainingly, the page that gets returned seems to only have the song title to [a Tom Jones song](https://www.azlyrics.com/lyrics/tomjones/itsnotunusual.html).) 

Whenever you call `requests.get` to retrieve a page, put a `time.sleep(5 + 10*random.random())` on the next line. This will help you not to get blocked. If you _do_ get blocked, which you can identify if the returned pages are not correct, just request a lyrics page through your browser. You'll be asked to perform a CAPTCHA and then your requests should start working again. 

## Part 1: Finding Links to Songs Lyrics

That general artist page has a list of all songs for that artist with links to the individual song pages. 

Q: Take a look at the `robots.txt` page on www.azlyrics.com. (You can read more about these pages [here](https://developers.google.com/search/docs/advanced/robots/intro).) Is the scraping we are about to do allowed or disallowed by this page? How do you know? 

A: When looking at www.azlyrics.com/robots.txt, I found the following:
User-agent: *
Disallow: /lyricsdb/
Disallow: /song/
Allow: /

User-agent: 008
Disallow: /


Since User-agent shows Allow: / it indicates general scrapping is allowed. There are some specific directories that disallow crawling, to include /lyricsdb/ and /song/. The lyrics pages are not explicity disallowed as there is an Allow: / directive.

In [23]:
# Let's set up a dictionary of lists to hold our links
lyrics_pages = defaultdict(list)

for artist, artist_page in artists.items():
    print(f"Fetching artist page URL for {artist}: {artist_page}")
    # request the page and sleep
    r = requests.get(artist_page)
    time.sleep(5 + 10 * random.random())

    # Check if request was successful
    if r.status_code == 200:
        print(f"Successfully fetched artist page for {artist}")
        # Parse the HTML content of the page
        soup = BeautifulSoup(r.content, 'html.parser')

        # Find all elements with the 'listalbum-item' class, which may contain song links
        song_items = soup.find_all('div', class_='listalbum-item')

        # Loop through each song item
        for item in song_items:
            # Find all links within the song item
            links = item.find_all('a', href=True)

            # Extract and append lyric page links
            for link in links:
                href = link['href']
                # Check if the link leads to a lyric page
                if href.startswith('../lyrics/') and href.endswith('.html'):
                    # Append the full URL to the list of links for the artist
                    full_url = f"https://www.azlyrics.com{href.replace('..', '')}"
                    lyrics_pages[artist].append(full_url)
                # Check if the link leads to a lyric page (alternative format)
                elif href.startswith('/lyrics/') and href.endswith('.html'):
                    # Append the full URL to the list of links for the artist
                    full_url = f"https://www.azlyrics.com{href}"
                    lyrics_pages[artist].append(full_url)

# Print the collected links for each artist
for artist, links in lyrics_pages.items():
    print(f"Links for {artist}:")
    for link in links:
        print(link)

Fetching artist page URL for taylorswift: https://www.azlyrics.com/t/taylorswift.html
Successfully fetched artist page for taylorswift
Fetching artist page URL for blackeyedpeas: https://www.azlyrics.com/b/blackeyedpeas.html
Successfully fetched artist page for blackeyedpeas
Links for taylorswift:
https://www.azlyrics.com/lyrics/taylorswift/samegirl.html
https://www.azlyrics.com/lyrics/taylorswift/babyblue.html
https://www.azlyrics.com/lyrics/taylorswift/neverfade.html
https://www.azlyrics.com/lyrics/taylorswift/fire.html
https://www.azlyrics.com/lyrics/taylorswift/maryjo.html
https://www.azlyrics.com/lyrics/taylorswift/myturntobeme.html
https://www.azlyrics.com/lyrics/taylorswift/kidinthecrowd.html
https://www.azlyrics.com/lyrics/taylorswift/iusedtofly.html
https://www.azlyrics.com/lyrics/taylorswift/luckyyou.html
https://www.azlyrics.com/lyrics/taylorswift/pointofview.html
https://www.azlyrics.com/lyrics/taylorswift/whydoyoutellme.html
https://www.azlyrics.com/lyrics/taylorswift/hous

Let's make sure we have enough lyrics pages to scrape. 

In [24]:
for artist, lp in lyrics_pages.items():
    assert len(lp) > 20, f"{artist} has less than 20 unique lyric links."

In [25]:
# Let's see how long it's going to take to pull these lyrics 
# if we're waiting `5 + 10*random.random()` seconds 

if not lyrics_pages:
    print("No lyrics pages found.")
else:
    for artist, links in lyrics_pages.items():
        print(f"For {artist} we have {len(links)} links.")
        print(f"The full pull will take for this artist will take {round(len(links)*10/3600,2)} hours.")


For taylorswift we have 465 links.
The full pull will take for this artist will take 1.29 hours.
For blackeyedpeas we have 171 links.
The full pull will take for this artist will take 0.47 hours.


In [26]:
# Let's see how long it's going to take to pull these lyrics 
# if we're waiting `5 + 10*random.random()` seconds 
for artist, links in lyrics_pages.items() : 
    print(f"For {artist} we have {len(links)}.")
    print(f"The full pull will take for this artist will take {round(len(links)*10/3600,2)} hours.")


For taylorswift we have 465.
The full pull will take for this artist will take 1.29 hours.
For blackeyedpeas we have 171.
The full pull will take for this artist will take 0.47 hours.


## Part 2: Pulling Lyrics

Now that we have the links to our lyrics pages, let's go scrape them! Here are the steps for this part. 

1. Create an empty folder in our repo called "lyrics". 
1. Iterate over the artists in `lyrics_pages`. 
1. Create a subfolder in lyrics with the artist's name. For instance, if the artist was Cher you'd have `lyrics/cher/` in your repo.
1. Iterate over the pages. 
1. Request the page and extract the lyrics from the returned HTML file using BeautifulSoup.
1. Use the function below, `generate_filename_from_url`, to create a filename based on the lyrics page, then write the lyrics to a text file with that name. 


In [27]:
def generate_filename_from_link(link) :
    
    if not link :
        return None
    
    # drop the http or https and the html
    name = link.replace("https","").replace("http","")
    name = link.replace(".html","")

    name = name.replace("/lyrics/","")
    
    # Replace useless chareacters with UNDERSCORE
    name = name.replace("://","").replace(".","_").replace("/","_")
    
    # tack on .txt
    name = name + ".txt"
    
    return(name)


In [31]:
import os
import shutil

# Check if the "lyrics" folder exists
if os.path.isdir("lyrics"):
    # If it exists, remove it and all its contents
    shutil.rmtree("lyrics/")

# Create a new "lyrics" folder
os.mkdir("lyrics")

In [34]:
# URL stub for AZLyrics
url_stub = "https://www.azlyrics.com"
# Start time
start = time.time()
# Counter for total pages
total_pages = 0

for artist, links in lyrics_pages.items():
    # Create subfolder for the artist
    artist_folder = os.path.join("lyrics", artist.lower())
    os.makedirs(artist_folder, exist_ok=True)

    # Counter for number of songs pulled
    songs_pulled = 0

    for link in links:
        if songs_pulled >= 20:
            break  # Limit to 20 songs per artist

        # Request the lyrics page and extract title and lyrics
        r = requests.get(link)
        time.sleep(5 + 10 * random.random())

        if r.status_code == 200:
            soup = BeautifulSoup(r.content, 'html.parser')
            title = soup.find('h1').text.strip()  # Extract title
            lyrics_div = soup.find('div', class_=None)
            lyrics = lyrics_div.text.strip() if lyrics_div else ""  # Extract lyrics

            # Write title and lyrics to a text file
            filename = os.path.join(artist_folder, generate_filename_from_link(link))
            with open(filename, 'w') as f:
                f.write(title + '\n\n')
                f.write(lyrics)
                songs_pulled += 1

                total_pages += 1

# End time
end = time.time()
print(f"Total pages pulled: {total_pages}")
print(f"Time taken: {end - start} seconds")

Total pages pulled: 40
Time taken: 476.29794359207153 seconds


In [35]:
print(f"Total run time was {round((time.time() - start)/3600,2)} hours.")

Total run time was 0.61 hours.


---

# Evaluation

This assignment asks you to pull data by scraping www.AZLyrics.com.  After you have finished the above sections , run all the cells in this notebook. Print this to PDF and submit it, per the instructions.

In [36]:
# Simple word extractor from Peter Norvig: https://norvig.com/spell-correct.html
def words(text): 
    return re.findall(r'\w+', text.lower())

## Checking Lyrics 

The output from your lyrics scrape should be stored in files located in this path from the directory:
`/lyrics/[Artist Name]/[filename from URL]`. This code summarizes the information at a high level to help the instructor evaluate your work. 

In [37]:
artist_folders = os.listdir("lyrics/")
artist_folders = [f for f in artist_folders if os.path.isdir("lyrics/" + f)]

for artist in artist_folders : 
    artist_files = os.listdir("lyrics/" + artist)
    artist_files = [f for f in artist_files if 'txt' in f or 'csv' in f or 'tsv' in f]

    print(f"For {artist} we have {len(artist_files)} files.")

    artist_words = []

    for f_name in artist_files : 
        with open("lyrics/" + artist + "/" + f_name) as infile : 
            artist_words.extend(words(infile.read()))

            
    print(f"For {artist} we have roughly {len(artist_words)} words, {len(set(artist_words))} are unique.")


For blackeyedpeas we have 20 files.
For blackeyedpeas we have roughly 60 words, 41 are unique.
For taylorswift we have 20 files.
For taylorswift we have roughly 79 words, 55 are unique.
